# Imports and setup

Constants

In [1]:
BATCH_SIZE = 50
EPOCHS = 50
SEED = 42
LEARNING_RATE = 1e-2
MAX_SENTENCE_WORDS = "unbounded"
NR_FROM_EACH_CLASS = 10000
N_QUBITS = 2
LAYERS = 1

Native Python libary imports

In [2]:
import random
from time import time

Library imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optax

import jax
import jax.numpy as jnp
from jax import jit, grad

Lambeq imports

In [ ]:
import lambeq
from lambeq import SpacyTokeniser
from lambeq import Dataset
from lambeq import QuantumTrainer, SPSAOptimizer

Code imports

In [ ]:
from linear_model import LinearModel
from optax_optimizer import OptaxOptimizer

**Make this notebook deterministic**

In [5]:
random.seed(SEED)
np.random.seed(SEED)

Set `numpy` options

In [6]:
np.set_printoptions(precision=3, linewidth = 200, threshold=100)

# Load data

In [7]:
# Load poetry dataset
df_train = pd.read_csv("poem_sentiment/train_tokenised.csv", index_col = 0)
df_dev = pd.read_csv("poem_sentiment/dev_tokenised.csv", index_col = 0)
df_test = pd.read_csv("poem_sentiment/test_tokenised.csv", index_col = 0)
df_list = [df_train, df_dev, df_test]

# Change sentiment to correct format and rename columns
for df in df_list:
    df["sentiment"] = df.sentiment.apply(lambda x: [0., 1.] if x == 1 else [1., 0.])
    df.rename({"sentence":"tokenised", "sentiment":"labels"}, inplace = True, axis=1)

# Truncate sentences if needed 
df_joined = pd.concat(df_list)
max_words = max(df["tokenised"].map(len)) if MAX_SENTENCE_WORDS == "unbounded" else MAX_SENTENCE_WORDS
for df in df_list:
    df["tokenised"] = [x[:max_words] for x in df["tokenised"]]

In [8]:
print("Nr. training sentences: ", len(df_train))
print("Nr. test sentences: ", len(df_test))
print("Nr. validation sentences: ", len(df_dev))

Nr. training sentences:  298
Nr. test sentences:  35
Nr. validation sentences:  39


# Set up model and train

In [9]:
from ansatz import _hardware_efficient_ansatz, _trivial_combine, _zero_ket, _multi_cnot_and_measure
hea = _hardware_efficient_ansatz(N_QUBITS, LAYERS)
initial_state = _zero_ket(N_QUBITS)
cnots = _multi_cnot_and_measure(N_QUBITS)
model = LinearModel.from_tokenised_sentences(df["tokenised"],
                                             initial_state,
                                             hea,
                                             _trivial_combine,
                                             2*N_QUBITS*LAYERS,
                                             0,
                                             end=cnots)

In [10]:
loss = lambda y_hat, y: -jnp.sum(y * jnp.log(y_hat)) / len(y)  # binary cross-entropy loss
acc = lambda y_hat, y: jnp.sum(jnp.round(y_hat) == y) / len(y) / 2  # half due to double-counting
eval_metrics = {"acc": acc}

trainer = QuantumTrainer(
    model,
    loss_function=loss,
    epochs=EPOCHS,
    optimizer=OptaxOptimizer.get(optax.adam),
    #optimizer=SPSAOptimizer,
    optim_hyperparams={'learning_rate': 1e-3},
    #optim_hyperparams={'a': 0.05, 'c': 0.06, 'A':0.01*EPOCHS},
    evaluate_functions=eval_metrics,
    evaluate_on_train=True,
    verbose = 'text',
    seed=SEED
)

In [11]:
train_dataset = Dataset(
            df_train["tokenised"],
            df_train.labels,
            batch_size=BATCH_SIZE)

val_dataset = Dataset(df_dev["tokenised"], df_dev["labels"])

In [12]:
# Activate NaN debugging just in case
from jax.config import config
config.update("jax_debug_nans", True)

In [13]:
time_start = time()
trainer.fit(train_dataset, val_dataset, evaluation_step=1, logging_step=5)
time_end = time()
elapsed = time_end - time_start

/home/gmatos/phd/projects/linear-model/optax_optimizer.py:107: UserWarning: OptaxOptimizer.state_dict not yet fully implemented
  warnings.warn('OptaxOptimizer.state_dict not yet fully implemented')
Epoch 1:   train/loss: 0.7875   valid/loss: 0.8437   train/acc: 0.5570   valid/acc: 0.5128
Epoch 5:   train/loss: 0.6083   valid/loss: 0.7770   train/acc: 0.6409   valid/acc: 0.5897
Epoch 10:  train/loss: 0.5155   valid/loss: 0.7271   train/acc: 0.7416   valid/acc: 0.5385
Epoch 15:  train/loss: 0.4769   valid/loss: 0.6888   train/acc: 0.7953   valid/acc: 0.6154
Epoch 20:  train/loss: 0.4547   valid/loss: 0.6940   train/acc: 0.8221   valid/acc: 0.6410
Epoch 25:  train/loss: 0.4372   valid/loss: 0.7143   train/acc: 0.8356   valid/acc: 0.6410
Epoch 30:  train/loss: 0.4225   valid/loss: 0.7449   train/acc: 0.8456   valid/acc: 0.6154
Epoch 35:  train/loss: 0.4106   valid/loss: 0.7863   train/acc: 0.8557   valid/acc: 0.5641
Epoch 40:  train/loss: 0.4010   valid/loss: 0.8038   train/acc: 0.8758   

In [14]:
test_acc = acc(model(df_test["tokenised"]), jnp.array(list(df_test["labels"])))
print("Test accuracy: ", test_acc)

Test accuracy:  0.4857142857142857


In [15]:
print("Time elapsed: ", elapsed)

Time elapsed:  26.224316835403442


In [16]:
results_df = pd.DataFrame({"train_epoch_costs": trainer.train_epoch_costs, "train_acc":trainer.train_results['acc'], "val_costs":trainer.val_costs, "val_acc":trainer.val_results['acc']})

In [17]:
results_df.to_csv(f"./data/{N_QUBITS=},{LAYERS=},{BATCH_SIZE=},{EPOCHS=},{SEED=},{LEARNING_RATE=},{MAX_SENTENCE_WORDS=},{NR_FROM_EACH_CLASS=}.csv")